## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [1]:
from keras import backend as K
from keras.models import Model
from keras.optimizers import Adam
from resnet_builder import resnet
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.layers import Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
# 讀取資料集並作前處理
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train / 255.
x_test = x_test / 255.
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [3]:
# 建立 ResNet 模型
model = resnet(input_shape=(32,32,3)) 
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 16)   64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 16)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [4]:
from keras.preprocessing.image import ImageDataGenerator

In [5]:
data_generator = ImageDataGenerator(rotation_range=20,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    horizontal_flip=True)

In [6]:
data_generator.fit(x_train)

In [7]:
batch_size = 32 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 30 # 訓練整個資料集共 30個循環

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model.fit_generator(data_generator.flow(x_train, y_train, batch_size=batch_size),
                              epochs=epochs,
                              steps_per_epoch=int(len(x_train)/32),
                              verbose=1,
                              validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/30
1562/1562 [==============================] - 116s 74ms/step - loss: 1.9096 - acc: 0.4627 - val_loss: 1.8163 - val_acc: 0.4861
Epoch 2/30
1562/1562 [==============================] - 110s 71ms/step - loss: 1.4642 - acc: 0.5847 - val_loss: 1.8900 - val_acc: 0.4629
Epoch 3/30
1562/1562 [==============================] - 109s 70ms/step - loss: 1.3064 - acc: 0.6307 - val_loss: 1.6883 - val_acc: 0.5365
Epoch 4/30
1562/1562 [==============================] - 109s 70ms/step - loss: 1.1970 - acc: 0.6658 - val_loss: 1.6308 - val_acc: 0.5633
Epoch 5/30
1562/1562 [==============================] - 108s 69ms/step - loss: 1.1165 - acc: 0.6950 - val_loss: 1.2742 - val_acc: 0.6811
Epoch 6/30
1562/1562 [==============================] - 109s 70ms/step - loss: 1.0570 - acc: 0.7142 - val_loss: 1.0793 - val_acc: 0.7156
Epoch 7/30
1562/1562 [==============================] - 108s 69ms/step - loss: 1.0143 - acc: 0.7293 - val_loss: 1.2650 - val_acc: 0.6677
Epoch 8/30
1562/1562 [===================